In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!java -version
!pip install pyspark

update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java to provide /usr/bin/java (java) in manual mode
openjdk version "1.8.0_222"
OpenJDK Runtime Environment (build 1.8.0_222-8u222-b10-1ubuntu1~18.04.1-b10)
OpenJDK 64-Bit Server VM (build 25.222-b10, mixed mode)
     |████████████████████████████████| 215.7MB 60kB/s 
     |████████████████████████████████| 204kB 50.5MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.4-py2.py3-none-any.whl size=216130387 sha256=3890d7271f1987cceeb1039e79cac6c2d9440139f7c5f51137a4dabfb9cdf49c
  Stored in directory: /root/.cache/pip/wheels/ab/09/4d/0d184230058e654eb1b04467dbc1292f00eaa186544604b471
Successfully built pyspark


In [0]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

In [0]:
from google.colab import files

In [18]:
files.upload()

Saving 2015-summary.csv to 2015-summary.csv


{'2015-summary.csv': b'DEST_COUNTRY_NAME,ORIGIN_COUNTRY_NAME,count\nUnited States,Romania,15\nUnited States,Croatia,1\nUnited States,Ireland,344\nEgypt,United States,15\nUnited States,India,62\nUnited States,Singapore,1\nUnited States,Grenada,62\nCosta Rica,United States,588\nSenegal,United States,40\nMoldova,United States,1\nUnited States,Sint Maarten,325\nUnited States,Marshall Islands,39\nGuyana,United States,64\nMalta,United States,1\nAnguilla,United States,41\nBolivia,United States,30\nUnited States,Paraguay,6\nAlgeria,United States,4\nTurks and Caicos Islands,United States,230\nUnited States,Gibraltar,1\nSaint Vincent and the Grenadines,United States,1\nItaly,United States,382\nUnited States,Federated States of Micronesia,69\nUnited States,Russia,161\nPakistan,United States,12\nUnited States,Netherlands,660\nIceland,United States,181\nMarshall Islands,United States,42\nLuxembourg,United States,155\nHonduras,United States,362\nThe Bahamas,United States,955\nUnited States,Senegal,4

In [19]:
!ls #see your directory

2015-summary.csv  sample_data


In [0]:
datafile='2015-summary.csv'

In [0]:
spark=(SparkSession
       .builder
       .master('local')
       .appName('flightsApp')
       .getOrCreate()
    
) #the session

In [22]:
spark

In [0]:
sc=spark.sparkContext

In [24]:
sc

<SparkContext master=local appName=flightsApp>

In [0]:
flightData2015=(spark
                .read
                .option('inferSchema', 'true') #try to enforce data expection of the datas
                .option('header','true')
                .csv(datafile)
                ) 

In [26]:
flightData2015

DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: int]

In [27]:
flightData2015.take(3) #prendre les 3premiers elements

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344)]

In [28]:
flightData2015.rdd

MapPartitionsRDD[16] at javaToPython at NativeMethodAccessorImpl.java:0

In [0]:
a=flightData2015.sort('count')

In [32]:
a.show(3)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|          Moldova|      United States|    1|
|    United States|          Singapore|    1|
|    United States|            Croatia|    1|
+-----------------+-------------------+-----+
only showing top 3 rows



In [33]:
flightData2015.sort('count').explain()

== Physical Plan ==
*(2) Sort [count#12 ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(count#12 ASC NULLS FIRST, 200)
   +- *(1) FileScan csv [DEST_COUNTRY_NAME#10,ORIGIN_COUNTRY_NAME#11,count#12] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/content/2015-summary.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:int>


In [0]:
spark.conf.set('spark.sql.shuffle.partitions',"5") #change the number of partitions by default to 200 to 5

In [35]:
flightData2015.sort('count').take(2) #sort c'est trier par le count

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Singapore', count=1),
 Row(DEST_COUNTRY_NAME='Moldova', ORIGIN_COUNTRY_NAME='United States', count=1)]

In [0]:
dataframeWay=flightData2015.groupby('DEST_COUNTRY_NAME').count()

In [41]:
dataframeWay.take(2)

[Row(DEST_COUNTRY_NAME='Moldova', count=1),
 Row(DEST_COUNTRY_NAME='Bolivia', count=1)]

In [42]:
dataframeWay.explain()

== Physical Plan ==
*(2) HashAggregate(keys=[DEST_COUNTRY_NAME#10], functions=[count(1)])
+- Exchange hashpartitioning(DEST_COUNTRY_NAME#10, 5)
   +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#10], functions=[partial_count(1)])
      +- *(1) FileScan csv [DEST_COUNTRY_NAME#10] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/content/2015-summary.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>


In [0]:
flightData2015.createOrReplaceTempView('flight_data_2015') #create a table from the data frame

In [0]:
sqlWay=spark.sql("" "SELECT DEST_COUNTRY_NAME, COUNT(1) FROM flight_data_2015 GROUP BY DEST_COUNTRY_NAME"
"")

In [53]:
sqlWay.take(5)

[Row(DEST_COUNTRY_NAME='Moldova', count(1)=1),
 Row(DEST_COUNTRY_NAME='Bolivia', count(1)=1),
 Row(DEST_COUNTRY_NAME='Algeria', count(1)=1),
 Row(DEST_COUNTRY_NAME='Turks and Caicos Islands', count(1)=1),
 Row(DEST_COUNTRY_NAME='Pakistan', count(1)=1)]

In [54]:
spark.sql("select max(count) from flight_data_2015").take(1)

[Row(max(count)=370002)]

In [0]:
from pyspark.sql.functions import max

In [58]:
flightData2015.select(max("count")).take(1)

[Row(max(count)=370002)]